# basic imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# data acquisition 

In [3]:
#data acquisition 
train_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv')
test_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv')
train_demo_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv')
test_demo_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv')

In [4]:
train_df.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0


✅ Core Metadata Columns (8)
	•	row_id: Unique row identifier
	•	sequence_id: ID for one full gesture sequence (Transition → Pause → Gesture)
	•	sequence_counter: Position within the sequence
	•	sequence_type: Type of gesture — target (BFRB-like) or non-target
	•	gesture: Gesture label (target variable)
	•	subject: Participant ID
	•	orientation: Body posture during recording
	•	behavior: Current phase (Transition, Pause, or Gesture)

⸻

📡 Sensor Columns (311 total)

🌀 Inertial Measurement Unit (IMU): 7 columns
	•	acc_x, acc_y, acc_z: Acceleration (m/s²)
	•	rot_w, rot_x, rot_y, rot_z: Orientation as quaternions

🌡️ Thermopile (Temperature): 5 columns
	•	thm_1 to thm_5: Temperature in °C from 5 infrared sensors

📏 Time-of-Flight (Distance): 299 columns
	•	tof_[1-5]_v[0-63]: Depth pixels from 5 ToF sensors (64 pixels each; 5 × 64 = 320 → some may be missing due to communication failures)


In [5]:
train_demo_df.head()

,subject,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SUBJ_000206,1,41,1,1,172.0,50,25.0
1,SUBJ_001430,0,11,0,1,167.0,51,27.0
2,SUBJ_002923,1,28,1,0,164.0,54,26.0
3,SUBJ_003328,1,33,1,1,171.0,52,25.0
4,SUBJ_004117,0,15,0,1,184.0,54,28.0


In [6]:
train_df = train_df[train_df['phase'] == 'Gesture']

In [7]:
train_df_merged = train_df.merge(on='subject',right=train_demo_df)

In [8]:
train_df_merged.head()

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000007_000025,Target,SEQ_000007,25,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,7.027344,...,90.0,86.0,89.0,0,12,1,1,163.0,52,24.0
1,SEQ_000007_000026,Target,SEQ_000007,26,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,6.984375,...,87.0,85.0,86.0,0,12,1,1,163.0,52,24.0
2,SEQ_000007_000027,Target,SEQ_000007,27,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,7.027344,...,92.0,86.0,92.0,0,12,1,1,163.0,52,24.0
3,SEQ_000007_000028,Target,SEQ_000007,28,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,6.488281,...,93.0,89.0,93.0,0,12,1,1,163.0,52,24.0
4,SEQ_000007_000029,Target,SEQ_000007,29,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Cheek - pinch skin,6.988281,...,97.0,92.0,-1.0,0,12,1,1,163.0,52,24.0


In [9]:
test_df_merged = test_df.merge(on='subject',right=test_demo_df)

In [10]:
test_df_merged.head()

,row_id,sequence_id,sequence_counter,subject,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000001_000000,SEQ_000001,0,SUBJ_055840,9.039062,5.261719,0.800781,0.367188,-0.397400,-0.629028,...,-1.0,111.0,-1.0,0,13,0,1,177.0,52,27.0
1,SEQ_000001_000001,SEQ_000001,1,SUBJ_055840,9.421875,3.460938,-1.113281,0.353882,-0.507141,-0.652710,...,211.0,187.0,178.0,0,13,0,1,177.0,52,27.0
2,SEQ_000001_000002,SEQ_000001,2,SUBJ_055840,10.160156,2.082031,-3.871094,0.384094,-0.532104,-0.639648,...,-1.0,197.0,177.0,0,13,0,1,177.0,52,27.0
3,SEQ_000001_000003,SEQ_000001,3,SUBJ_055840,9.773438,1.355469,-4.371094,0.387756,-0.531982,-0.634033,...,-1.0,197.0,183.0,0,13,0,1,177.0,52,27.0
4,SEQ_000001_000004,SEQ_000001,4,SUBJ_055840,9.195312,1.011719,-3.222656,0.382751,-0.534180,-0.638367,...,-1.0,200.0,173.0,0,13,0,1,177.0,52,27.0


# •	Test set doesn’t contain:gesture, sequence_type, orientation, behavior, phase

⸻


# Label : Gesture

In [11]:
#dropping rest of columns
train_df_merged.drop(['sequence_type', 'orientation', 'behavior', 'phase'],axis=1,inplace=True)

In [12]:
train_df_merged.head()

,row_id,sequence_id,sequence_counter,subject,gesture,acc_x,acc_y,acc_z,rot_w,rot_x,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000007_000025,SEQ_000007,25,SUBJ_059520,Cheek - pinch skin,7.027344,5.523438,4.312500,0.254578,-0.285583,...,90.0,86.0,89.0,0,12,1,1,163.0,52,24.0
1,SEQ_000007_000026,SEQ_000007,26,SUBJ_059520,Cheek - pinch skin,6.984375,5.640625,4.656250,0.257019,-0.281921,...,87.0,85.0,86.0,0,12,1,1,163.0,52,24.0
2,SEQ_000007_000027,SEQ_000007,27,SUBJ_059520,Cheek - pinch skin,7.027344,5.488281,5.613281,0.241577,-0.280518,...,92.0,86.0,92.0,0,12,1,1,163.0,52,24.0
3,SEQ_000007_000028,SEQ_000007,28,SUBJ_059520,Cheek - pinch skin,6.488281,5.832031,4.425781,0.216675,-0.307922,...,93.0,89.0,93.0,0,12,1,1,163.0,52,24.0
4,SEQ_000007_000029,SEQ_000007,29,SUBJ_059520,Cheek - pinch skin,6.988281,6.023438,3.660156,0.209595,-0.315918,...,97.0,92.0,-1.0,0,12,1,1,163.0,52,24.0


In [13]:
test_df_merged.head()

,row_id,sequence_id,sequence_counter,subject,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,...,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000001_000000,SEQ_000001,0,SUBJ_055840,9.039062,5.261719,0.800781,0.367188,-0.397400,-0.629028,...,-1.0,111.0,-1.0,0,13,0,1,177.0,52,27.0
1,SEQ_000001_000001,SEQ_000001,1,SUBJ_055840,9.421875,3.460938,-1.113281,0.353882,-0.507141,-0.652710,...,211.0,187.0,178.0,0,13,0,1,177.0,52,27.0
2,SEQ_000001_000002,SEQ_000001,2,SUBJ_055840,10.160156,2.082031,-3.871094,0.384094,-0.532104,-0.639648,...,-1.0,197.0,177.0,0,13,0,1,177.0,52,27.0
3,SEQ_000001_000003,SEQ_000001,3,SUBJ_055840,9.773438,1.355469,-4.371094,0.387756,-0.531982,-0.634033,...,-1.0,197.0,183.0,0,13,0,1,177.0,52,27.0
4,SEQ_000001_000004,SEQ_000001,4,SUBJ_055840,9.195312,1.011719,-3.222656,0.382751,-0.534180,-0.638367,...,-1.0,200.0,173.0,0,13,0,1,177.0,52,27.0


In [14]:
train_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255817 entries, 0 to 255816
Columns: 344 entries, row_id to elbow_to_wrist_cm
dtypes: float64(334), int64(6), object(4)
memory usage: 671.4+ MB


In [15]:
#object columns (categorical)
for col in train_df_merged.columns:
    if train_df_merged[col].dtype != 'int' and train_df_merged[col].dtype != 'float':
        print(col)

row_id
sequence_id
subject
gesture


In [16]:
#handling gesture 
train_df_merged['gesture'].unique()

array(['Cheek - pinch skin', 'Forehead - pull hairline',
       'Write name on leg', 'Feel around in tray and pull out an object',
       'Neck - scratch', 'Neck - pinch skin', 'Eyelash - pull hair',
       'Eyebrow - pull hair', 'Forehead - scratch',
       'Above ear - pull hair', 'Wave hello', 'Write name in air',
       'Text on phone', 'Pull air toward your face',
       'Pinch knee/leg skin', 'Scratch knee/leg skin',
       'Drink from bottle/cup', 'Glasses on/off'], dtype=object)

In [17]:
train_df[train_df.phase == 'Gesture']['gesture'].unique()

array(['Cheek - pinch skin', 'Forehead - pull hairline',
       'Write name on leg', 'Feel around in tray and pull out an object',
       'Neck - scratch', 'Neck - pinch skin', 'Eyelash - pull hair',
       'Eyebrow - pull hair', 'Forehead - scratch',
       'Above ear - pull hair', 'Wave hello', 'Write name in air',
       'Text on phone', 'Pull air toward your face',
       'Pinch knee/leg skin', 'Scratch knee/leg skin',
       'Drink from bottle/cup', 'Glasses on/off'], dtype=object)

In [18]:
train_df_merged['tof_5_v51'].isna().sum()

13628

In [19]:
gesture_map = {'Above ear - pull hair':0,
"Forehead - pull hairline":1,
"Forehead - scratch":2,
"Eyebrow - pull hair":3,
"Eyelash - pull hair":4,
"Neck - pinch skin"	:5,
'Neck - scratch':6,
"Cheek - pinch skin":7,
"Drink from bottle/cup"	:8,
"Glasses on/off":8,
"Pull air toward your face"	:8,
"Pinch knee/leg skin":8,
"Scratch knee/leg skin":8,
"Write name on leg"	:8,
"Text on phone"	:8,
"Feel around in tray and pull out an object":8,
"Write name in air":8,
"Wave hello":8}
train_df_merged['gesture'] = train_df_merged['gesture'].replace(gesture_map)

In [20]:
train_df_merged['gesture'].unique()

array([7, 1, 8, 6, 5, 4, 3, 2, 0])

In [21]:
from collections import defaultdict

inv_map_full = defaultdict(list)
for k, v in gesture_map.items():
    inv_map_full[v].append(k)

dict(inv_map_full)

{0: ['Above ear - pull hair'],
 1: ['Forehead - pull hairline'],
 2: ['Forehead - scratch'],
 3: ['Eyebrow - pull hair'],
 4: ['Eyelash - pull hair'],
 5: ['Neck - pinch skin'],
 6: ['Neck - scratch'],
 7: ['Cheek - pinch skin'],
 8: ['Drink from bottle/cup',
  'Glasses on/off',
  'Pull air toward your face',
  'Pinch knee/leg skin',
  'Scratch knee/leg skin',
  'Write name on leg',
  'Text on phone',
  'Feel around in tray and pull out an object',
  'Write name in air',
  'Wave hello']}

In [22]:
train_df_merged.drop(['row_id','sequence_id','subject'],axis=1,inplace=True)
test_df_merged.drop(['row_id','sequence_id','subject'],axis=1,inplace=True)

In [23]:
X_train = train_df_merged.drop('gesture',axis=1)
y_train = train_df_merged['gesture']
X_test = test_df_merged

### Scaling

In [24]:
from sklearn.preprocessing import StandardScaler
st_sclr = StandardScaler()
X_train_scl = st_sclr.fit_transform(X_train)
X_test_scl = st_sclr.transform(X_test)

In [25]:
from catboost import CatBoostClassifier
cat_clf = CatBoostClassifier(task_type='GPU')
cat_clf.fit(X_train,y_train)

Learning rate set to 0.201141
0:	learn: 1.7667917	total: 14.9s	remaining: 4h 8m 21s
1:	learn: 1.6262158	total: 15s	remaining: 2h 4m 21s
2:	learn: 1.5236491	total: 15s	remaining: 1h 23m 1s
3:	learn: 1.4446410	total: 15s	remaining: 1h 2m 21s
4:	learn: 1.3870070	total: 15.1s	remaining: 49m 56s
5:	learn: 1.3346127	total: 15.1s	remaining: 41m 39s
6:	learn: 1.2913092	total: 15.1s	remaining: 35m 45s
7:	learn: 1.2561289	total: 15.2s	remaining: 31m 19s
8:	learn: 1.2207829	total: 15.2s	remaining: 27m 52s
9:	learn: 1.1897244	total: 15.2s	remaining: 25m 6s
10:	learn: 1.1626599	total: 15.2s	remaining: 22m 50s
11:	learn: 1.1430909	total: 15.3s	remaining: 20m 57s
12:	learn: 1.1179246	total: 15.3s	remaining: 19m 22s
13:	learn: 1.1013256	total: 15.3s	remaining: 17m 59s
14:	learn: 1.0792685	total: 15.4s	remaining: 16m 48s
15:	learn: 1.0649290	total: 15.4s	remaining: 15m 46s
16:	learn: 1.0491888	total: 15.4s	remaining: 14m 51s
17:	learn: 1.0356181	total: 15.4s	remaining: 14m 2s
18:	learn: 1.0233218	total

In [26]:
import joblib
joblib.dump(cat_clf, "catboost_model.pkl")

['catboost_model.pkl']

In [27]:
import joblib
cat_clf = joblib.load("/kaggle/input/cmi-sensor-private/catboost_model.pkl")

In [28]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(device='gpu')
lgbm.fit(X_train,y_train)
joblib.dump(lgbm, "lgbm_model.pkl")

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 84057
[LightGBM] [Info] Number of data points in the train set: 255817, number of used features: 340
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 334 dense feature groups (81.97 MB) transferred to GPU in 0.078536 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score -2.564402
[LightGBM] [Info] Start training from score -2.555654
[LightGBM] [Info] Start training from score -2.551083
[LightGBM] [Info] Start training from score -2.571538
[LightGBM] [Info] Start training from score -2.571385
[LightGBM] [Info] Start training from score -2.555302
[LightGBM] [Info] Start training from score -2.548330
[LightGBM] [Info] Start training from score -2.570465
[LightGBM] [Info] Start training from score -0.961896


['lgbm_model.pkl']

In [29]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(tree_method='gpu_hist',  # GPU acceleration
    predictor='gpu_predictor')

In [30]:
xgb_clf.fit(X_train,y_train)
joblib.dump(xgb_clf, "xgb_model.pkl")

['xgb_model.pkl']